# Blackholes

Synthesizer has a collection of routines for modelling the emission of black holes (i.e. AGN).
When coupled with a `Galaxy` object it also allows users to combine with the emission from other galaxy components (i.e. stellar emission).

Unlike stellar emission the division between a `particle` and a `parametric` `BlackHoles`/`BlackHole` object is not well defined; we explain below the differences in different use cases.

## Parametric Blackholes

If you are only interested in exploring the parameter space of AGN emission without simulation data to forward model you can use a `parametric.BlackHole`. There are some specific differences compared to `particle.BlackHole` objects to keep in mind:

- A `parametric.BlackHole` can only ever describe a singular black hole. 

- A `parametric.BlackHole`'s "position" (i.e. if making an image) is described by a `PointSource` morphology object rather than coordinates.

- A `parametric.BlackHole` exists in isolation, i.e. it does not interface directly with other parametric components.

In [ ]:
import numpy as np
from unyt import Mpc, Msun, deg, yr

from synthesizer import galaxy
from synthesizer.parametric import BlackHole

# Set a random number seed to ensure consistent results
np.random.seed(42)

First we need to initialise our `BlackHole` object with the parameters that will be needed to compute spectra.

In [ ]:
blackhole = BlackHole(
    mass=1e8 * Msun,
    inclination=60 * deg,
    accretion_rate=1 * Msun / yr,
    metallicity=0.01,
)

Like other synthesizer objects we can get more information using the `print` command.

In [ ]:
print(blackhole)

## Particle blackholes

### Creating Particle Blackholes

Before generating some simple observational quantities from physical properties we first need to create a `BlackHoles` object. This object can be found in `synthesizer/particle/blackholes.py`. 

In [ ]:
from synthesizer.particle import BlackHoles, Gas

Lets create an instance of `BlackHoles` containing 4 fake black holes.
To do so we can provide a number of optional keyword arguments, but for now lets just provide their masses, metallicities, coordinates and accretion rates (the parameters required for spectra calculation).
Note that `masses` and `accretion_rates` are positional arguments, and must therefore always be provided for `particle.BlackHoles`, while `parametric.BlackHole`s have more flexibility.

In [ ]:
# Make fake properties
n = 4
masses = 10 ** np.random.uniform(low=7, high=9, size=n) * Msun
coordinates = np.random.normal(0, 1.5, (n, 3)) * Mpc
accretion_rates = 10 ** np.random.uniform(low=-2, high=1, size=n) * Msun / yr
metallicities = np.full(n, 0.01)

# And get the black holes object
bh = BlackHoles(
    masses=masses,
    coordinates=coordinates,
    accretion_rates=accretion_rates,
    metallicities=metallicities,
)

For some emission models we require an inclination. This could, in principle, be calculated from the simulation and passed at instantiation, but for now we can use an in-built method to generate random inclinations.

In [ ]:
bh.calculate_random_inclination()
print(bh.inclination)

#### Blackhole properties

On initialisation a handful of properties will automatically be calculated if their prerequisites are met.
For example, if `masses` and `accretion_rates` are provided, `bolometric_luminosities` are automatically calculated using, $$L_{\rm \bullet, bol} = \epsilon_{r}\dot{M}_{\bullet}c^{2}.$$
Note that the radiative efficency (`epsilon`) defaults to 0.1, but can be passed as an array with a value for each particle. 

In [ ]:
bh.bolometric_luminosities

Here are some more examples of calculated properties.

In [ ]:
print(bh.eddington_ratio)
print(bh.accretion_rate_eddington)
print(bh.eddington_luminosity)

As with most synthesizer objects a summary of the object can be printed using `print`.

In [ ]:
print(bh)

#### Calculating black hole metallicity

If we want to calculate emission from the black hole and its surroundings we need to know the metallicity of the gas surrounding the black hole.
In the example above we could have passed an array of metallicities at instantiation, but most of the time we will not know ahead of time what these values should be.
Instead, we can use the gas surrounding the black hole to calculate what this metallicity is. To do this we need to first create a `Galaxy` with both a `Gas` component and `BlackHoles`, again using fake data.

In [ ]:
# Make fake gas properties
ngas = 200
ms = np.full(ngas, 10**6.5)  # Msun
pos = np.random.normal(0, 1.5, (ngas, 3))  # cMpc
hsml = np.full(ngas, 0.75)  # cMpc
metals = np.full(ngas, 0.01)

# And make the gas object
gas = Gas(
    masses=ms * Msun,
    metallicities=metals,
    coordinates=pos * Mpc,
    smoothing_lengths=hsml * Mpc,
)

# And now create the galaxy
galaxy = galaxy(stars=None, gas=gas, black_holes=bh)

Now we have the galaxy we can use `galaxy.calculate_black_hole_metallicity()` to calculate the black holes' metallicity.
This method will find all gas particles with smoothing lengths that intersect the black hole and calculate the mass--weighted average of their metallicity.
If a black hole does not find any gas neighbours then a default metallicity is set instead.
This defaults to solar metallicity (0.012) but can be overwritten by passing a new `default_metallicity` as shown below.

In [ ]:
galaxy.calculate_black_hole_metallicity(default_metallicity=0.07)
print("Z_BH =", galaxy.black_holes.metallicities)